<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/089_ParseResponse_ExecuteAction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🎯 Major Concepts to Focus On

### 1️⃣ Parsing the LLM’s Response into a **Structured Action**

* **Why:** Free-form text is ambiguous; a structured format (e.g., JSON inside a markdown block) lets you *reliably* figure out what to do next.
* **What to learn:**

  * Define a clear **action schema** (e.g., `{"tool_name": "...", "args": {...}}`).
  * Extract that schema from the LLM’s raw output.
  * Validate it — if it’s missing fields or malformed, produce an **error action** so the agent can recover.

---

### 2️⃣ **Executing the Action**

* **Why:** This is where decisions turn into *real effects* — reading files, calling APIs, updating data.
* **What to learn:**

  * Map `tool_name` → the right function.
  * Pass `args` into that function.
  * Handle unknown tools or bad arguments gracefully.
  * Think of this step as the **“hands”** of your agent; parsing was the **“eyes”**.

---

### 3️⃣ **Updating Memory with Results**

* **Why:** The agent needs to remember *both* what it intended to do (assistant output) and what happened (user feedback with action results).
* **What to learn:**

  * Store the LLM’s structured action output as an `assistant` message.
  * Store the action’s **result** as a `user` message.
  * This ensures the LLM “knows” the outcome of its previous step on the next turn.

---

### 4️⃣ **Deciding Whether to Continue**

* **Why:** You need an exit strategy so the loop doesn’t run forever.
* **What to learn:**

  * Check if `tool_name` is `"terminate"`.
  * End after a set max iterations or other stopping conditions.
  * Provide a clear final message on termination.

---

## 🧩 How These Fit Together

In the **Agent Loop**:

1. **Memory window** → LLM → *structured action* (parsed from text).
2. **Execute** that action (call the matching tool).
3. **Update memory** with the action + result.
4. **Decide to continue or stop**.

This is the shift from *purely conversational* to **goal-oriented** agents that can:

* Plan a step
* Take an action
* Observe the outcome
* Iterate until the task is complete




# Parse Response


### **Purpose**

`parse_action` takes the **raw text** from the LLM and converts it into a **structured Python dictionary** that the rest of your agent can actually *use*.

Without this, the agent would be stuck with free-form text that might be inconsistent or hard to interpret.

---

### **Step-by-step**

```python
response = extract_markdown_block(response, "action")
```

* Pulls out **only** the contents inside a markdown code block starting with \`\`\`action.
* Example raw output from the LLM:

  ````
  ```action
  {"tool_name": "list_files", "args": {}}
  ````

  ````
  After extraction:  
  ```json
  {"tool_name": "list_files", "args": {}}
  ````

---

```python
response_json = json.loads(response)
```

* Converts the JSON string into a **Python dictionary**.
* If the LLM output isn’t valid JSON, this will raise a `json.JSONDecodeError` and jump to the `except` block.

---

```python
if "tool_name" in response_json and "args" in response_json:
    return response_json
else:
    return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
```

* Ensures the two required keys are present:

  * `"tool_name"` — which tool to run
  * `"args"` — parameters for that tool
* If either is missing, it returns a special **error action** telling the LLM what went wrong.

---

```python
except json.JSONDecodeError:
    return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}
```

* If the JSON was broken (bad syntax, missing quotes, etc.), return an **error action** saying the JSON is invalid.

---

### **Why this matters**

1. **Consistency** — The agent’s execution code can rely on always getting a dictionary with the same structure.
2. **Safety** — You detect and handle malformed output instead of letting your code break.
3. **Control** — You can feed the `error` action back to the LLM so it knows to fix its formatting on the next turn.

---

**Example:**

````python
raw = """```action
{"tool_name": "list_files", "args": {}}
```"""
parsed = parse_action(raw)
# parsed -> {"tool_name": "list_files", "args": {}}
````

Now your agent can do:

```python
if parsed["tool_name"] == "list_files":
    run list_files()
```



In [ ]:
def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

# This parsing step is critical to ensuring the response is actionable. It provides a structured output, such as:

{
    "tool_name": "list_files",
    "args": {}
}

# By breaking down the LLM’s output into tool_name and args,
# the agent can precisely determine the next action and its inputs.

In [8]:
# --- 1. Install dependencies ---
!pip install --quiet python-dotenv openai

import os, json, re
from dotenv import load_dotenv
from openai import OpenAI

# --- 1) Load API key from /content/API_KEYS.env ---
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

# --- 2) System prompt + chat history ---
system_instructions = (
    """
You are an AI file assistant.
Respond ONLY with a JSON action inside a markdown block, like:

```action
{"tool_name": "list_files", "args": {}}
```

Available tools:
- list_files: returns list of files in /content/files
- read_file: args={"file_name": "<filename>"}

Rules:
- Respond with exactly one JSON object inside a ```action block.
- If unsure, return an error action instead of prose.
"""
).strip()

messages = [{"role": "system", "content": system_instructions}]

def add_message(role, content):
    messages.append({"role": role, "content": content})

# --- 3) Extract JSON from a ```action code block ---
def extract_markdown_block(text, block_name):
    pattern = rf"```{block_name}\s*(.*?)\s*```"
    match = re.search(pattern, text, flags=re.DOTALL | re.IGNORECASE)
    return match.group(1) if match else None

# --- 4) Parse the model's response into an action dict ---
def parse_action(response_text):
    action_str = extract_markdown_block(response_text, "action")
    if not action_str:
        return {"tool_name": "error", "args": {"message": "No action block found."}}
    try:
        action_json = json.loads(action_str)
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON."}}
    if isinstance(action_json, dict) and "tool_name" in action_json and "args" in action_json:
        return action_json
    return {"tool_name": "error", "args": {"message": "Missing tool_name or args."}}

# --- 5) Tools ---
FILES_DIR = "/content/files"

def list_files():
    try:
        return sorted(os.listdir(FILES_DIR))
    except FileNotFoundError:
        return [f"Error: Folder not found: {FILES_DIR}"]

def read_file(file_name, max_chars=2000):
    path = os.path.join(FILES_DIR, file_name)
    if not os.path.isfile(path):
        return f"Error: File not found: {file_name}"
    try:
        with open(path, "r", encoding="utf-8", errors="replace") as f:
            content = f.read(max_chars + 1)
        if len(content) > max_chars:
            content = content[:max_chars] + "\n... [truncated]"
        return content
    except Exception as e:
        return f"Error reading {file_name}: {e}"

# --- 6) Execute action ---

def execute_action(action):
    tool = (action or {}).get("tool_name")
    args = (action or {}).get("args", {})

    if tool == "list_files":
        return {"result": list_files()}
    elif tool == "read_file":
        fname = args.get("file_name")
        if not fname:
            return {"error": "Missing arg: file_name"}
        return {"result": read_file(fname)}
    elif tool == "error":
        return {"error": args.get("message", "Unknown error")}
    elif tool == "terminate":
        return {"result": args.get("message", "Terminated")}
    else:
        return {"error": f"Unknown tool: {tool}"}

# --- 7) Full step: prompt -> parse -> execute ---

def step(user_input, memory_size=None):
    add_message("user", user_input)
    visible = messages if memory_size is None else messages[-memory_size:]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=visible,
    )
    reply = response.choices[0].message.content
    print("Raw model output:\n", reply)

    action = parse_action(reply)
    print("\nParsed action:", action)

    result = execute_action(action)
    print("\nExecution result (JSON):", result)

    # Update history so the model sees outcomes next turn
    add_message("assistant", reply)              # the model's action choice
    add_message("user", json.dumps(result))      # the tool result as feedback
    return result

In [9]:
# --- 8) Demo ---
_ = step("List all files in the /content/files directory.")

Raw model output:
 ```action
{"tool_name": "list_files", "args": {}}
```

Parsed action: {'tool_name': 'list_files', 'args': {}}

Execution result (JSON): {'result': ['000_Prompting for Agents -GAIL.txt', '001_PArse_the Response.txt', '002_Execute_the_Action.txt', '003_gent Feedback and Memory.txt']}


In [10]:
_ = step("Read the file named '001_PArse_the Response.txt'.")

Raw model output:
 ```action
{"tool_name": "read_file", "args":{"file_name":"001_PArse_the Response.txt"}}
```

Parsed action: {'tool_name': 'read_file', 'args': {'file_name': '001_PArse_the Response.txt'}}

Execution result (JSON): {'result': '\n#==========Parse the Response\n\nAfter generating a response, the next step is to extract the intended action and its parameters from the LLM’s output. The response is expected to follow a predefined structure, such as a JSON format encapsulated within a markdown code block. This structure ensures the action can be parsed and executed without ambiguity.\n\nIn the code, this is accomplished by locating and extracting the content between the ```action markers. If the response does not include a valid action block, the agent defaults to a termination action, returning the raw response as the message:\n\n#==============\n\ndef parse_action(response: str) -> Dict:\n    """Parse the LLM response into a structured action dictionary."""\n    try:\n   



## 🔑 Why returning a specific format matters

1. **Reliability**

   * Free-form text is hard for code to work with — the model could change wording, punctuation, or order at any time.
   * A strict JSON schema gives you something you can parse without guesswork.

2. **Automation**

   * If the output is always `{"tool_name": "...", "args": {...}}`, your agent loop can directly map `tool_name` to a function and pass `args` without human intervention.
   * This is what lets agents run unattended.

3. **Error handling**

   * When the model violates the format, you can detect it and return an `error` action — keeping the loop safe.
   * Without a schema, you might try to run the wrong tool or crash the agent.

4. **Interoperability**

   * Other parts of your system (UI, logging, APIs) can consume this structured output without needing to “understand” natural language.
   * You can swap models without changing the rest of the pipeline.

5. **Future expansion**

   * You can add more tools simply by adding more `tool_name` values — the model just picks the right one and you’re done.
   * The execution layer stays the same.

---

The key point:

> **Structured output turns the LLM into a decision-maker you can reliably hook into code** — rather than just a conversational partner.



In [12]:
# !pip -q install openai python-dotenv
import os, json, re
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path

# ---- Setup (safe to re-run) ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

FILES_DIR = Path("/content/files")

# ---- Tools ----

def list_files():
    try:
        return sorted([p.name for p in FILES_DIR.iterdir() if p.is_file()])
    except FileNotFoundError:
        return [f"Error: Folder not found: {FILES_DIR}"]

def read_file(file_name, max_chars=2400):
    target = (FILES_DIR / file_name).resolve()
    if FILES_DIR not in target.parents and target != FILES_DIR:
        return f"Error: {file_name} is outside {FILES_DIR}"
    if not target.exists() or not target.is_file():
        return f"Error: File not found: {file_name}"
    try:
        with open(target, "r", encoding="utf-8", errors="replace") as f:
            content = f.read(max_chars + 1)
        if len(content) > max_chars:
            content = content[:max_chars] + "\n... [truncated]"
        return content
    except Exception as e:
        return f"Error reading {file_name}: {e}"

# ---- Action parsing ----

def extract_markdown_block(text, block_name):
    pattern = rf"```{block_name}\s*(.*?)\s*```"
    m = re.search(pattern, text, flags=re.DOTALL | re.IGNORECASE)
    return m.group(1) if m else None

def parse_action(response_text):
    action_str = extract_markdown_block(response_text, "action")
    if not action_str:
        return {"tool_name": "error", "args": {"message": "No action block found."}}
    try:
        action_json = json.loads(action_str)
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON."}}
    if isinstance(action_json, dict) and "tool_name" in action_json and "args" in action_json:
        return action_json
    return {"tool_name": "error", "args": {"message": "Missing tool_name or args."}}

# ---- Execute action ----

def execute_action(action):
    tool = (action or {}).get("tool_name")
    args = (action or {}).get("args", {})

    if tool == "list_files":
        return {"result": list_files()}
    elif tool == "read_file":
        fname = args.get("file_name")
        if not fname:
            return {"error": "Missing arg: file_name"}
        return {"result": read_file(fname)}
    elif tool == "error":
        return {"error": args.get("message", "Unknown error")}
    elif tool == "terminate":
        return {"result": args.get("message", "Terminated")}
    else:
        return {"error": f"Unknown tool: {tool}"}

# ---- System prompt for multi-step behavior ----
SYSTEM_V3 = (
    """
You are an AI file agent. You can take multiple steps to complete the user's task.
Respond ONLY with a single JSON action inside a markdown block like:

```action
{"tool_name": "list_files", "args": {}}
```

Available tools:
- list_files: returns list of files in /content/files
- read_file: args={"file_name": "<filename>"}
- terminate: args={"message": "<final summary or reason>"}
- error: args={"message": "<what went wrong>"}

Rules:
1) Use tool(s) as needed. When done, return a terminate action with a clear message.
2) After each tool result is returned to you (as a user message), decide the next step.
3) No prose outside the JSON action block.
"""
).strip()

# ---- Agent loop (Feedback + Memory) ----

def run_agent(task: str, max_iters: int = 5, memory_size=None):
    messages = [{"role": "system", "content": SYSTEM_V3},
                {"role": "user", "content": task}]

    for i in range(1, max_iters + 1):
        visible = messages if memory_size is None else messages[-memory_size:]
        resp = client.chat.completions.create(model="gpt-4o-mini", messages=visible)
        reply = resp.choices[0].message.content
        print(f"\n=== Iteration {i}: Raw model output ===\n{reply}")

        action = parse_action(reply)
        print("Parsed action:", action)

        result = execute_action(action)
        print("Execution result:", (str(result)[:400] + ("..." if len(str(result))>400 else "")))

        # Update memory: assistant (the action) + user (the result)
        messages.append({"role": "assistant", "content": reply})
        messages.append({"role": "user", "content": json.dumps(result)})

        # Check for termination
        if action.get("tool_name") == "terminate":
            print("\nAgent terminated:", result.get("result"))
            return {"messages": messages, "final": result}

    # Safety stop
    print("\nMax iterations reached. Terminating.")
    return {"messages": messages, "final": {"error": "max_iters_reached"}}

In [13]:
# ---- Demo calls ----
# Example 1: Ask it to pick and read a file, then summarize and terminate
out = run_agent("Find a file that mentions 'Parse' in the name, read it, then give me a 2-sentence summary and terminate.")


=== Iteration 1: Raw model output ===
```action
{"tool_name": "list_files", "args": {}}
```
Parsed action: {'tool_name': 'list_files', 'args': {}}
Execution result: {'result': ['000_Prompting for Agents -GAIL.txt', '001_PArse_the Response.txt', '002_Execute_the_Action.txt', '003_gent Feedback and Memory.txt']}

=== Iteration 2: Raw model output ===
```action
{"tool_name": "read_file", "args":{"file_name": "001_PArse_the Response.txt"}}
```
Parsed action: {'tool_name': 'read_file', 'args': {'file_name': '001_PArse_the Response.txt'}}
Execution result: {'result': '\n#==========Parse the Response\n\nAfter generating a response, the next step is to extract the intended action and its parameters from the LLM’s output. The response is expected to follow a predefined structure, such as a JSON format encapsulated within a markdown code block. This structure ensures the action can be parsed and executed without ambiguity.\n\nIn the code, this is accomp...

=== Iteration 3: Raw model output ===

In [14]:
# Example 2: Just list files, then stop
out = run_agent("List the files and then terminate with a short description of what you see.")


=== Iteration 1: Raw model output ===
```action
{"tool_name": "list_files", "args": {}}
```
Parsed action: {'tool_name': 'list_files', 'args': {}}
Execution result: {'result': ['000_Prompting for Agents -GAIL.txt', '001_PArse_the Response.txt', '002_Execute_the_Action.txt', '003_gent Feedback and Memory.txt']}

=== Iteration 2: Raw model output ===
```action
{"tool_name": "terminate", "args":{"message": "Files listed: 000_Prompting for Agents -GAIL.txt, 001_PArse_the Response.txt, 002_Execute_the_Action.txt, 003_gent Feedback and Memory.txt."}}
```
Parsed action: {'tool_name': 'terminate', 'args': {'message': 'Files listed: 000_Prompting for Agents -GAIL.txt, 001_PArse_the Response.txt, 002_Execute_the_Action.txt, 003_gent Feedback and Memory.txt.'}}
Execution result: {'result': 'Files listed: 000_Prompting for Agents -GAIL.txt, 001_PArse_the Response.txt, 002_Execute_the_Action.txt, 003_gent Feedback and Memory.txt.'}

Agent terminated: Files listed: 000_Prompting for Agents -GAIL.tx

How the parsing + feedback loop self-corrects bad model output.

# How the loop “teaches” the model to fix itself

1. Model replies (maybe with bad formatting).
2. `parse_action(...)` tries to parse.
3. If parsing fails, you create an **error action**.
4. You then feed that **error** back to the model as a **`user`** message (feedback).
5. On the next turn, the model sees the error and (usually) fixes its format.

Tiny, minimal demo (no external calls), just to see the flow:

````python
# Minimal illustration of the feedback loop using fake replies:

messages = []

def add_message(role, content):
    messages.append({"role": role, "content": content})

def parse_action(response_text):
    import json, re
    m = re.search(r"```action\s*(.*?)\s*```", response_text, re.DOTALL|re.IGNORECASE)
    if not m:
        return {"tool_name": "error", "args": {"message": "No action block found."}}
    try:
        data = json.loads(m.group(1))
        if "tool_name" in data and "args" in data:
            return data
        return {"tool_name": "error", "args": {"message": "Missing tool_name or args."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON."}}

# 1) Bad model reply (no code fence at all)
bad_reply = '{"tool_name": "list_files", "args": {}}'  # missing ```action ... ```
add_message("assistant", bad_reply)
action = parse_action(bad_reply)
print("Parsed:", action)  # -> error

# 2) Feed the error back as *user* feedback
add_message("user", json.dumps(action))

# 3) Next (improved) model reply, now correctly wrapped
fixed_reply = """```action
{"tool_name": "list_files", "args": {}}
```"""
add_message("assistant", fixed_reply)
print("Parsed:", parse_action(fixed_reply))  # -> valid action dict
````

What to notice:

* The first parse fails → returns `{"tool_name":"error", "args":{"message":"..."}}`.
* That error is appended as a **user** message (feedback the model will see next turn).
* The second reply is corrected and parses cleanly.





## 🔑 Key Points to Carry Forward

1. **Formatting is a contract**

   * The model’s “contract” with the agent is: *I’ll always return this JSON schema inside a code block*.
   * Parsing enforces the contract and protects your code from unstructured surprises.

2. **Memory enables self-correction**

   * By saving the **error message** in the chat history, you give the model a chance to “see its mistake” and fix it.
   * Without memory, the model wouldn’t know what went wrong last time.

3. **Error messages are just instructions**

   * An “error” in the agent loop is just another message to the LLM — you can phrase it however is most useful for guiding correction.

4. **Parse early, execute later**

   * You always want to parse the model’s intent before running tools.
   * This separation makes it easy to handle errors, log intentions, or even override them.

5. **Graceful degradation**

   * If parsing fails multiple times, you can fall back to a safe termination instead of letting the loop spin forever.

